In [43]:
# Importing Python packages

import pandas as pd
import numpy as np
import plotnine as p9
import os
import geopandas as gpd
from pathlib import Path


In [44]:
# Reading in baseline TAZ data

sharepoint_path = f'C:/Users/{os.environ.get( "USERNAME" )}/Buro Happold'
project_path = Path(f"{sharepoint_path}/P060927 Plan Capital 2040 - 02_Documents/11_GIS/05 Urban Models")
ium_path = os.path.join(project_path,'ium_ad.gpkg')
zone_taz_2022 = gpd.read_file(ium_path, layer='zone_taz_2022')
zone_taz_2022.head(1)


,Z,SHAPE_Length,SHAPE_Area,NUM_OF_BUILDINGS,Calc_Total_Ex_GFA,Ex_GFA,RES_GFA,Residential_Labour_GFA,OFFICE_GFA,RETAIL_GFA,...,RETMAL_PRO,MEDCLI_PRO,MEDHOS_PRO,NONTER_PRO,TER_PRO,SCHPLCS_NT,SCHPLCS_TE,SCHPLCS_PR,SCHPLCS_SE,geometry
0,1044,47079.0,606869.0,679.0,297016.0,297542.0,271410.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((270352.336 2727199.707, 269735..."


In [45]:
# Reading in taz zone delta

zone_taz_delta = gpd.read_file(ium_path, layer='zone_taz_delta')
zone_taz_delta.head(1)

,Z,ZONE,EPA,UPC_PRECIN,PRECINCT_N,ST_DIST_ID,ST_DIST_EN,UDM_DIST_I,UDM_DIST_E,REGION,...,score_component_rank_sum_rank_mp,score_component_rank_mean_rank_mp,score_component_rank_sum_rank_urban,score_component_rank_mean_rank_urban,score_development_priority,2022_taz_far,gfa_per_ha,landmodel_limit_gfa_per_ha,landmodel_limit_gfa_per_ha_delta,geometry
0,2510,2510,None,1.0,Abu Dhabi,1.0,Abu Dhabi West Coast And Islands And Mussaffah...,500.0,ABU AL ABYAD ISLAND,ADM,...,NaN,NaN,1.0,1.0,5.0,0.000003,0.026657,0.026657,-0.025877,"MULTIPOLYGON (((182767.838 2687320.951, 183001..."


In [46]:
# Reading in land-use split

land_use_crosswalk = pd.read_csv('taxonomy_crosswalks.csv')
land_use_crosswalk.head(1)

,CROSSWALK,VARIABLE_1,VARIABLE_2,NOTES
0,TAZ to MP land use crosswalk,Residential (GFA),RES_GFA,NaN


In [61]:
tmp_2040_select_columns = ['Z','MP_ProjectID','RES_GFA','RETAIL_GFA','OFFICE_GFA','L_IND_GFA','M_IND_GFA','H_IND_GFA','SCHOOL_GFA','MED_GFA','OTHER_GFA']

In [62]:
tmp_2040_input = pd.read_csv('20240605_TAZ_TMP_2040.csv')


In [73]:
tmp_2040 = tmp_2040_input[tmp_2040_select_columns]
tmp_2040 = tmp_2040[~tmp_2040['Z'].isna()]
tmp_2040['Z'] = tmp_2040['Z'].astype(str)
tmp_2040['TOTAL_GFA'] = tmp_2040.select_dtypes(include=[float, int]).sum(axis=1)
tmp_2040.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3476 entries, 0 to 3475
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Z             3476 non-null   object 
 1   MP_ProjectID  3476 non-null   object 
 2   RES_GFA       3476 non-null   float64
 3   RETAIL_GFA    3476 non-null   float64
 4   OFFICE_GFA    3476 non-null   float64
 5   L_IND_GFA     3476 non-null   float64
 6   M_IND_GFA     3476 non-null   float64
 7   H_IND_GFA     3476 non-null   float64
 8   SCHOOL_GFA    3476 non-null   float64
 9   MED_GFA       3476 non-null   float64
 10  OTHER_GFA     3476 non-null   float64
 11  TOTAL_GFA     3476 non-null   float64
dtypes: float64(10), object(2)
memory usage: 353.0+ KB


In [82]:
tmp_2040_per = tmp_2040.select_dtypes(include=[float, int]).div(tmp_2040['TOTAL_GFA'], axis=0)
tmp_2040_per = pd.concat([tmp_2040[['Z','MP_ProjectID']],tmp_2040_per],axis = 1)
tmp_2040_per = tmp_2040_per[tmp_2040['MP_ProjectID'] != '0'].reset_index(drop = True)
tmp_2040_per = tmp_2040_per[~tmp_2040['MP_ProjectID'].isna()].reset_index(drop = True)
tmp_2040_per

C:\Users\jtindale\AppData\Local\Temp\ipykernel_26720\1306470590.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,Z,MP_ProjectID,RES_GFA,RETAIL_GFA,OFFICE_GFA,L_IND_GFA,M_IND_GFA,H_IND_GFA,SCHOOL_GFA,MED_GFA,OTHER_GFA,TOTAL_GFA
0,1.0,1007.42.4,0.986446,0.001520,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012034,1.0
1,2.0,1007.42.4,0.899572,0.007761,0.000000,0.000000,0.000000,0.000000,0.075361,0.000000,0.017306,1.0
2,3.0,1007.0,0.439153,0.000000,0.560847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
3,4.0,1007.27,0.700105,0.022941,0.000000,0.000000,0.000000,0.000000,0.162524,0.000000,0.114430,1.0
4,5.0,1007.42.4,0.990072,0.000953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008975,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1649,3472.0,1997.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
1650,3473.0,1997.0,0.000000,0.873139,0.076865,0.000000,0.000000,0.000000,0.000000,0.000000,0.049995,1.0
1651,3474.0,1995.0,0.000000,0.006808,0.031058,0.207263,0.377436,0.377436,0.000000,0.000000,0.000000,1.0
1652,3475.0,1995.0,0.000000,0.002353,0.000000,0.199269,0.398537,0.398537,0.000000,0.000858,0.000445,1.0


In [83]:
tmp_2040_per

,Z,MP_ProjectID,RES_GFA,RETAIL_GFA,OFFICE_GFA,L_IND_GFA,M_IND_GFA,H_IND_GFA,SCHOOL_GFA,MED_GFA,OTHER_GFA,TOTAL_GFA
0,1.0,1007.42.4,0.986446,0.001520,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012034,1.0
1,2.0,1007.42.4,0.899572,0.007761,0.000000,0.000000,0.000000,0.000000,0.075361,0.000000,0.017306,1.0
2,3.0,1007.0,0.439153,0.000000,0.560847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
3,4.0,1007.27,0.700105,0.022941,0.000000,0.000000,0.000000,0.000000,0.162524,0.000000,0.114430,1.0
4,5.0,1007.42.4,0.990072,0.000953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008975,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1649,3472.0,1997.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
1650,3473.0,1997.0,0.000000,0.873139,0.076865,0.000000,0.000000,0.000000,0.000000,0.000000,0.049995,1.0
1651,3474.0,1995.0,0.000000,0.006808,0.031058,0.207263,0.377436,0.377436,0.000000,0.000000,0.000000,1.0
1652,3475.0,1995.0,0.000000,0.002353,0.000000,0.199269,0.398537,0.398537,0.000000,0.000858,0.000445,1.0
